In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import xgboost as xgb 
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
print("######读取特征（基于EEG）######")
#EEG特征表
eegFeatureDF = pickle.load(open("./dump_file/eegFeatureDF","rb"))

if True:
    #加上早期的统计特征
    for eegCH in range(1,33,1):
        file_path = "./dump_file/CH{}_eeg_feat_df".format(eegCH)
        tmpDF = pickle.load(open(file_path,"rb"))
        eegFeatureDF = pd.concat([eegFeatureDF,tmpDF],axis=1)
if True:#加上样本熵特征
    filePath = "./dump_file_sampEn/sampEnFeatures".format(eegCH)
    tmpDF = pickle.load(open(filePath,"rb"))
    eegFeatureDF = pd.concat([eegFeatureDF,tmpDF],axis=1)
if False:
    #GSR特征表
    GSR_feature_df = pickle.load(open("./dump_file/df_feat_selected","rb"))
    eegFeatureDF = pd.concat([eegFeatureDF,GSR_feature_df],axis=1)

#总的特征向量样本大小
print("eegFeatureDF.shape:",eegFeatureDF.shape)

######读取特征（基于EEG）######
eegFeatureDF.shape: (1280, 1440)


In [3]:
if False:
    print("######数据缩放处理，归一化处理######")
    min_max_scaler = MinMaxScaler()
    eegFeatureDF = min_max_scaler.fit_transform(eegFeatureDF)


In [4]:
#featureDF = pd.concat([eegFeatureDF,GSR_feature_df],axis=1)
#featureDF = GSR_feature_df
featureDF = eegFeatureDF

In [5]:
#############################下面用愉悦度（效价）做分类#################################
if False:
    print("----------------这是高低愉悦度度二分类预测----------------")
    data = featureDF
    target = all_df_y_2c #高低愉悦度
    
    print("######xgb classification CV######")
    xgb_model = xgb.XGBClassifier(max_depth=5,learning_rate=0.1,n_estimators=50,
                                      objective='binary:logistic',booster='gbtree',n_jobs=10,
                                      subsample=0.9, colsample_bytree=0.9, colsample_bylevel=0.9,
                                      reg_alpha=0.5, reg_lambda=1.0,gamma=0,
                                      scale_pos_weight=1)
    xgb_scores = cross_val_score(xgb_model,data,target,cv=5,scoring='accuracy')
    print("xgb_scores:",abs(xgb_scores))
    print("xgb_scores_mean:",abs(xgb_scores.mean()))


In [6]:
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y['2cArousal'] = 0
all_df_y['2cArousal'][all_df_y['valence'] >= 5] = 1
all_df_y['2cValence'] = 0
all_df_y['2cValence'][all_df_y['valence'] >= 5] = 1
print(all_df_y.head(5))
y = all_df_y[["2cValence"]]
xTrainIdx = pickle.load(open("./dump_file/xTrainIdx","rb"))
xTestIdx = pickle.load(open("./dump_file/xTestIdx","rb"))
trainY = y.loc[xTrainIdx]
testY = y.loc[xTestIdx]
trainX = eegFeatureDF.loc[xTrainIdx]
testX = eegFeatureDF.loc[xTestIdx]

       valence  arousal  dominance  liking  2cArousal  2cValence
s01_0     7.71     7.60       6.90    7.83          1          1
s01_1     8.10     7.31       7.28    8.47          1          1
s01_2     8.58     7.54       9.00    7.08          1          1
s01_3     4.94     6.01       6.12    8.06          0          0
s01_4     6.96     3.92       7.19    6.05          1          1


In [7]:
if True:
    data = featureDF
    target = y
    print("######xgb classification ######")
    xgb_model = xgb.XGBClassifier(max_depth=5,learning_rate=0.1,n_estimators=50,
                                      objective='binary:logistic',booster='gbtree',n_jobs=10,
                                      subsample=0.9, colsample_bytree=0.9, colsample_bylevel=0.9,
                                      reg_alpha=0.5, reg_lambda=1.0,gamma=0,
                                      scale_pos_weight=1)
    xgb_model.fit(trainX.values,trainY.values)
    predY = xgb_model.predict(testX.values)
    print("ACC",accuracy_score(y_true=testY, y_pred=predY))
    print("F1",f1_score(y_true=testY, y_pred=predY))
    print("Recal",recall_score(y_true=testY,y_pred=predY))
    print("Precision",precision_score(y_true=testY, y_pred=predY))

######xgb classification ######
ACC 0.6536458333333334
F1 0.712742980561555
Recal 0.7603686635944701
Precision 0.6707317073170732
